In [5]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist
import sys
sys.path.append('/scratch/jho5ze/bionets/covid/scripts')
import entropy_utils as eu
from screed import ScreedDB
msadb = ScreedDB("../data/msa_0408/usa_msa_0408.fasta")

In [7]:
metadata = pd.read_csv("../data/metadata_2021_04_08.tsv", sep="\t", parse_dates = ["Collection date", "Submission date"])

metadata = metadata[["Accession ID", \
                   "Collection date", \
                   "Submission date", \
                   "Location", \
                   "Additional location information", \
                   "Sequence length", \
                   "Host", \
                   "AA Substitutions", \
                   "Is reference?"]]
def get_nth_slash(row, n):
    try:
        return row.split("/")[n].strip()
    except:
        return np.nan
    
for i in range(4):
    metadata[f"Location_{i}"] = metadata["Location"].apply(lambda row: get_nth_slash(row, i))
metadata = metadata.rename(columns={"Location_1":"country", "Location_2":"state"})

/apps/software/standard/compiler/gcc/9.2.0/jupyter_conda/2019.10-py3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (13,16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [58]:
def map_chars(line):
    char_map = {"A":0, "C":1, "T":2, "G":3}
    chars = []
    for c in line:
        if c in char_map:
            chars.append(char_map[c])
        else:
            chars.append(4)
            
    return chars

In [74]:
samples = metadata[metadata.country == "USA"]["Accession ID"].head(120).to_numpy()
sample_arr = []
for line in eu.msa_from_screed_ids(samples):
    if line[0] == ">":
        continue
    else:
        line_arr = map_chars(line)
#         print(line[1450:1460])
        sample_arr.append(line_arr)
sample_arr = np.array(sample_arr)

In [75]:
x = pdist(sample_arr, position_difference)
sum(x) / x.shape[0]

0.9917899176623811

In [40]:
# samples = ["ATCGA", "ATCGN", "ATCCA", "CTCGA"]
# sample_arr = []
# for sample in samples:
#     line_arr = [char_map[c] for c in sample]
#     sample_arr.append(line_arr)
# sample_arr = np.array(sample_arr)    
# print(sample_arr)

[[0 2 1 3 0]
 [0 2 1 3 4]
 [0 2 1 1 0]
 [1 2 1 3 0]]


In [63]:
def position_difference(u, v):
    return sum(u == v) / u.shape[0]

def calculate_jaccard_from_accessions(accessions):
    sample_arr = []
    for line in eu.msa_from_screed_ids(accessions):
        if line[0] == ">":
            continue
        else:
            line_arr = map_chars(line)
            sample_arr.append(line_arr)
    sample_arr = np.array(sample_arr)
    x = pdist(sample_arr, position_difference)
    return sum(x) / x.shape[0]

In [79]:
calculate_jaccard_from_accessions(["EPI_ISL_1222555"])

ZeroDivisionError: division by zero

In [71]:
from tqdm import tqdm

In [78]:
x = [1,2,3,4,5]
a, b = x[1:3]
x[3:]

[4, 5]

In [73]:
top_k = 10
top_k_sequenced_us = metadata[metadata.country == "USA"].Location_3.value_counts()[:top_k].index
top_state_county = []
for i in top_k_sequenced_us:
    top_state = metadata[(metadata.country == "USA") & (metadata.Location_3 == i)].state.value_counts().index[0]
    top_state_county.append((top_state, i))

output = []
for state, location in top_state_county:
    print(location)
        
    location_metadata = metadata[(metadata.country == "USA") & (metadata.state == state) & (metadata.Location_3 == location)]
    location_metadata = location_metadata[["Accession ID", "Collection date"]]
    location_metadata.columns = ["seq", "date"]
    location_metadata = location_metadata.dropna()
    
    date_window = 7
    date_step_size = 7

    for date in tqdm(pd.date_range(location_metadata.date.min(), location_metadata.date.max(), freq=f"{date_step_size}D")):
        end_date = date + np.timedelta64(date_window - 1, 'D')
        sub_seqs = location_metadata[(location_metadata.date >= date) & (location_metadata.date <= end_date)].seq
        sub_seqs = [s for s in sub_seqs if s in msadb]
        experiment_name = f"window_{date_window}_step_{date_step_size}"
        stdate = date.strftime("%Y-%m-%d")
        stend_date = end_date.strftime("%Y-%m-%d")
        output.append((stdate, stend_date, location, len(sub_seqs), calculate_jaccard_from_accessions(sub_seqs)))
#         print(stdate, stend_date, location, len(sub_seqs))
#         break
#     break

  0%|          | 0/51 [00:00<?, ?it/s]

Houston


  4%|▍         | 2/51 [01:33<38:16, 46.86s/it]


KeyboardInterrupt: 